In [1]:
# std imports
import copy
import math
import re

# installed package imports
import pyrealsense2 as rs
import numpy as np
import cv2
from pyzbar import pyzbar

In [2]:
class rotation_estimator:
    def __init__(self):
        self.theta = [0.0,0.0,0.0]
        self.alpha = 0.98
        self.first = True
        self.last_ts_gyro = 0.0
    
    # Function to calculate the change in angle of motion based on data from gyro
    # gyro_data - list of 3 elems
    # ts - float arrival time of curr gyro frame
    def process_gyro(self, gyro_data, ts):
        if self.first:
            self.last_ts_gyro = ts
            return
        
        gyro_angle = [gyro_data.x,gyro_data.y,gyro_data.z]
        
        dt_gyro = (ts-self.last_ts_gyro)/1000.0
        self.last_ts_gyro = ts
        
        gyro_angle = [gyro_angle[0]*dt_gyro,gyro_angle[1]*dt_gyro,gyro_angle[2]*dt_gyro]
        
        self.theta = [self.theta[0]-gyro_angle[2],self.theta[1]-gyro_angle[1],self.theta[2]+gyro_angle[0]]
    
    # Function to calculate the change in angle of motion based on data from accelerometer
    # accel_data - list of 3 elems
    def process_accel(self, accel_data):
        accel_angle = [0.0,0.0,0.0]
        accel_angle[2] = np.arctan2(accel_data.y,accel_data.z)
        accel_angle[0] = np.arctan2(accel_data.x, np.sqrt(accel_data.y**2+accel_data.z**2))
        
        if self.first:
            self.first = False
            self.theta = accel_angle
            self.theta[1] = np.pi/2
        else:
            self.theta[0] = self.theta[0]*self.alpha + accel_angle[0]*(1-self.alpha)
            self.theta[2] = self.theta[2]*self.alpha + accel_angle[2]*(1-self.alpha)
        
    def get_theta(self):
        theta_out = [0.0,0.0,0.0]
        theta_out = [self.theta[0]*180/np.pi, self.theta[1]*180/np.pi, (self.theta[2]-np.pi/2)*180/np.pi]
        return theta_out
    
def get_robot_euler_angle(frames):
    accel_data = gyro_data = gyro_ts = None
    
    accel_frame = frames.first_or_default(rs.stream.accel)
    if accel_frame:
        accel_data = accel_frame.as_motion_frame().get_motion_data()

    gyro_frame = frames.first_or_default(rs.stream.gyro)
    if gyro_frame:
        gyro_data = gyro_frame.as_motion_frame().get_motion_data()
        gyro_ts = gyro_frame.as_motion_frame().get_timestamp()
    
    return accel_frame, accel_data, gyro_frame, gyro_data, gyro_ts

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth stream
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30) # color stream
config.enable_stream(rs.stream.gyro) # gyro stream
config.enable_stream(rs.stream.accel) # accelerometer stream
align = rs.align(rs.stream.color) # align both streams to same pov

# Enable visualizer and filters for later use
colorizer = rs.colorizer()
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 3)
hole_filling = rs.hole_filling_filter()
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

# Image detection size
expected = 300
inScaleFactor = 0.007843
meanVal = 127.53

net = cv2.dnn.readNetFromTensorflow('frozen_inference_graph.pb', 'graph.pbtxt')  # pretrained net

swapRB = True
classNames = { 0: 'background',
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
    7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
    13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
    18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
    24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
    32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
    37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
    41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
    46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
    51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
    56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
    61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
    67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
    75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
    80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
    86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush' }

In [4]:
def get_robot_in_world_pos(robot_theta, color_image, depth_image, depth_frame, depth_intrin):
    # detect and decode barcodes
    object_depth = np.asanyarray(depth_frame.get_data())
    barcode_poses = []
    barcodes = pyzbar.decode(color_image)

    bar_x = []
    bar_y = []
    robo_x = []
    robo_y = []

    for barcode in barcodes:
        (x, y, w, h) = barcode.rect
        cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.rectangle(depth_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type

        curr = copy.deepcopy(object_depth)
        curr = curr[math.floor(((x)+(x+w))/2-1):math.ceil(((x)+(x+w))/2+1), math.floor(((y)+(y+h))/2-1):math.ceil(((y)+(y+h))/2+1)].astype(float)
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        curr = curr * depth_scale
        dist,_,_,_ = cv2.mean(curr)
        barcode_poses.append((barcodeData, dist))

        # draw the barcode data and barcode type on the image
        text = "{} ({})".format(barcodeData, dist)
        cv2.putText(color_image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # store poses of codes
        curr_pose = re.split(',|\(|\)',barcodeData)
        bar_x.append(float(curr_pose[1]))
        bar_y.append(float(curr_pose[2]))

        # Get 3d point of object detected
        relative_depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int((x+(x+w))/2), int((y+(y+h))/2)], dist)
#             print("\Relative Point: \nX ",relative_depth_point[0],"\nY ",relative_depth_point[1],"\nZ ",relative_depth_point[2])

        # Get world position of cam according to this barcode
        curr_cam_theta = robot_theta
#             print("Cam Theta: \n",curr_cam_theta)
        curr_cam_phi = np.arctan2(relative_depth_point[2],relative_depth_point[0])*180/np.pi
#             print("Cam Phi: \n",curr_cam_phi)
        curr_cam_alpha = 90 - curr_cam_theta + curr_cam_phi
#             print("Cam Alpha: \n",curr_cam_alpha)
        curr_cam_delta_x = dist*np.cos(np.radians(curr_cam_alpha))
#             print("Cam delta_x: \n",curr_cam_delta_x)
        curr_cam_delta_y = dist*np.sin(np.radians(curr_cam_alpha))
#             print("Cam delta_y: \n",curr_cam_delta_y)             
#             print("\nBarcode Pose: \nX ",curr_pose[1],"\nY ",curr_pose[2])
        robo_x.append(float(curr_pose[1])-float(curr_cam_delta_x))
        robo_y.append(float(curr_pose[2])-float(curr_cam_delta_y))

    if barcodes:
        curr_cam_pos = [np.average(robo_x), np.average(robo_y)]

        text_pos_x = 'X: ' + str(curr_cam_pos[0])
        text_pos_y = 'Y: ' + str(curr_cam_pos[1])

        cv2.putText(depth_image, text_pos_x, (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(depth_image, text_pos_y, (20, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        return curr_cam_pos
    
    return None
            

In [5]:
def get_human_in_world_pos(net, color_frame, depth_frame, profile, robot_theta, depth_image, color_image):
    expected = 300
    inScaleFactor = 0.007843
    meanVal = 127.53
    
    color_human_image = np.asanyarray(color_frame.get_data())
    
    # crop color image for detection
    height, width = color_human_image.shape[:2]
    
    aspect = width / height
    resized_color_image = cv2.resize(color_human_image, (round(expected * aspect), expected))
    crop_start = round(expected * (aspect - 1) / 2)
    crop_color_img = resized_color_image[0:expected, crop_start:crop_start + expected]

    # Perform object detection through net
    blob = cv2.dnn.blobFromImage(crop_color_img, inScaleFactor, (expected, expected), meanVal, False)
    net.setInput(blob)
    detections = net.forward("detection_out")

    label = detections[0, 0, 0, 1]
    conf = detections[0, 0, 0, 2]
    xmin = detections[0, 0, 0, 3]
    ymin = detections[0, 0, 0, 4]
    xmax = detections[0, 0, 0, 5]
    ymax = detections[0, 0, 0, 6]

    if (conf >= .5):
        className = classNames[int(label)]

        # Calculate box coordinates of detected object
        scale = height / expected
        xmin_depth = int((xmin * expected + crop_start) * scale)
        ymin_depth = int((ymin * expected) * scale)
        xmax_depth = int((xmax * expected + crop_start) * scale)
        ymax_depth = int((ymax * expected) * scale)
        xmin_depth, ymin_depth, xmax_depth, ymax_depth

        # Calculate depth of object
        depth = np.asanyarray(depth_frame.get_data())
        # Crop depth data:
        depth = depth[math.floor((xmax_depth + xmin_depth) / 2 - 1):math.ceil((xmax_depth + xmin_depth) / 2 + 1), math.floor((ymax_depth + ymin_depth) / 2 - 1):math.ceil((ymax_depth + ymin_depth) / 2 + 1)].astype(float)

        # Get data scale from the device and convert to meters
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        depth = depth * depth_scale
        dist, _, _, _ = cv2.mean(depth)

        # Get relative location of person to camera
        relative_depth_point = rs.rs2_deproject_pixel_to_point(depth_intrin, [int((xmin_depth + xmax_depth) / 2), int((ymin_depth + ymax_depth) / 2)], dist)

        # Get world position of cam according to this barcode
        curr_cam_theta = robot_theta
#             print("Cam Theta: \n",curr_cam_theta)
        curr_cam_phi = np.arctan2(relative_depth_point[2],relative_depth_point[0])*180/np.pi
#             print("Cam Phi: \n",curr_cam_phi)
        curr_cam_alpha = 90 - curr_cam_theta + curr_cam_phi
#             print("Cam Alpha: \n",curr_cam_alpha)
        curr_person_delta_x = dist*np.cos(np.radians(curr_cam_alpha))
#             print("Cam delta_x: \n",curr_cam_delta_x)
        curr_person_delta_y = dist*np.sin(np.radians(curr_cam_alpha))
#             print("Cam delta_y: \n",curr_cam_delta_y)             
#             print("\nBarcode Pose: \nX ",curr_pose[1],"\nY ",curr_pose[2])

        # Draw square on depth and color streams
        cv2.rectangle(depth_image, (xmin_depth, ymin_depth), (xmax_depth, ymax_depth), (255, 255, 255), 2)
        cv2.rectangle(color_image, (xmin_depth, ymin_depth), (xmax_depth, ymax_depth), (255, 255, 255), 2)
        cv2.putText(color_image, className + " @ " + "{:.2f}".format(dist) + "meters away", (xmin_depth, ymin_depth), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255))

        return curr_person_delta_x, curr_person_delta_y, color_frame.get_timestamp()
    
    return None, None, None

In [9]:
# State Representation
robot_x = 0
robot_y = 0
robot_theta = 90
robot_vx = 0
robot_vy = 0
human_x = 0
human_y = 0
human_theta = 0  # doesn't really matter
human_vx = 0
human_vy = 0
goal_x = 0  # set by user
goal_y = 0  # set by user
time_from_last_step = 0

# Start streaming
algo = rotation_estimator()
profile = pipeline.start(config)
first_run = True

# print("STARTED PIPELINE\n")

try:
    while True:
        if first_run:
#             print("THINKS ITS THE FIRST RUN\n")
            try:
                frames = pipeline.wait_for_frames()
                
                accel_frame, accel_data, gyro_frame, gyro_data, gyro_ts = get_robot_euler_angle(frames)
                if not accel_frame or not gyro_frame:
#                     print("FIRST NOT FOUND")
                    break
                    
                algo.process_gyro(gyro_data, gyro_ts)
                algo.process_accel(accel_data)
                first_run = False
#                 print("FIRST PROCESSED")
                continue

            except:
                print("EXCEPTION")
                break

#         print("NOT FIRST RUN ANY MORE\n")
                
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        accel_frame, accel_data, gyro_frame, gyro_data, gyro_ts = get_robot_euler_angle(frames)

        if gyro_frame:
            algo.process_gyro(gyro_data, gyro_ts)
        if accel_frame:
            algo.process_accel(accel_data)
            
        robot_theta = algo.get_theta()[1]

        if not depth_frame or not color_frame:
            # @TODO GET NEXT ACTION BASED ON JUST THE LAST STATE'S VALUES OR MAYBE STOP IN CURR POSITION
            print("DIDNT GET DEPTH OR COLOR FRAME\n")
            break

#         print("GOT DEPTH AND COLOR FRAME\n")
            
        # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
        depth_frame = depth_to_disparity.process(depth_frame)
        depth_frame = spatial.process(depth_frame)
        depth_frame = disparity_to_depth.process(depth_frame)
        depth_frame = hole_filling.process(depth_frame)

        # get intrinsics
        depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
        color_intrin = color_frame.profile.as_video_stream_profile().intrinsics

        # Convert images to numpy arrays
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        color_image = np.asanyarray(color_frame.get_data())

        curr_cam_pos = get_robot_in_world_pos(robot_theta, color_image, depth_image, depth_frame, depth_intrin)
        if curr_cam_pos:
            robot_x = curr_cam_pos[0]
            robot_y = curr_cam_pos[1]
        
        # FIND POSITION OF PERSON
        person_first_scan = None
        person_second_scan = None
        found_first = False
        found_second = False

        curr_person_delta_x, curr_person_delta_y, frame_timestamp = get_human_in_world_pos(net, color_frame, depth_frame, profile, robot_theta, depth_image, color_image)
        
        if curr_person_delta_x and curr_person_delta_y:
            person_first_scan = [(float(robot_x + curr_person_delta_x)), (float(robot_y + curr_person_delta_y)), frame_timestamp]
            
            human_x = person_first_scan[0]
            human_y = person_first_scan[1]
            found_first = True                                            

            # Stack both images horizontally
            images = np.hstack((color_image, depth_image))

            # Show images
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images)
            key = cv2.waitKey(1)

            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                break

            print("GOT FIRST PERSON DATA AS: X: ", human_x, " Y: ", human_y, "\n")

            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            frames = align.process(frames)

            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()

            accel_frame, accel_data, gyro_frame, gyro_data, gyro_ts = get_robot_euler_angle(frames)

            if gyro_frame:
                algo.process_gyro(gyro_data, gyro_ts)
            if accel_frame:
                algo.process_accel(accel_data)
            
            robot_theta = algo.get_theta()[1]
            
            if not depth_frame or not color_frame:
                # @TODO GET NEXT ACTION BASED ON JUST THE LAST STATE'S VALUES OR MAYBE STOP IN CURR POSITION
                print("DIDNT GET DEPTH OR COLOR FRAME\n")
                break
            
            # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
            depth_frame = depth_to_disparity.process(depth_frame)
            depth_frame = spatial.process(depth_frame)
            depth_frame = disparity_to_depth.process(depth_frame)
            depth_frame = hole_filling.process(depth_frame)

            # get intrinsics
            depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
            color_intrin = color_frame.profile.as_video_stream_profile().intrinsics

            # Convert images to numpy arrays
            depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
            color_image = np.asanyarray(color_frame.get_data())
            
            curr_person_delta_x, curr_person_delta_y, frame_timestamp = get_human_in_world_pos(net, color_frame, depth_frame, profile, robot_theta, depth_image, color_image)
            
            if curr_person_delta_x and curr_person_delta_y:
                person_second_scan = [(float(robot_x + curr_person_delta_x)), (float(robot_y + curr_person_delta_y)), frame_timestamp]

                human_x = person_second_scan[0]
                human_y = person_second_scan[1]
                found_second = True
                
                # Stack both images horizontally
                images = np.hstack((color_image, depth_image))

                # Show images
                cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
                cv2.imshow('RealSense', images)
                key = cv2.waitKey(1)

                # Press esc or 'q' to close the image window
                if key & 0xFF == ord('q') or key == 27:
                    cv2.destroyAllWindows()
                    break

        if found_first and found_second:
            human_vx = (person_second_scan[0] - person_first_scan[0]) / (person_second_scan[2] - person_first_scan[2])
            human_vy = (person_second_scan[1] - person_first_scan[1]) / (person_second_scan[2] - person_first_scan[2])
            print("GOT PERSON VELOCITY DATA AS: VX: ", human_vx, " VY: ", human_vy, "\n")
            
        elif found_first and not found_second:
            human_vx = 0
            human_vy = 0
        
        else:
            human_x = None
            human_y = None
            human_vx = None
            human_vy = None
        
        if human_x and human_y:
            text_pos_x = 'human X: ' + str(human_x)
            text_pos_y = 'human Y: ' + str(human_y)
            cv2.putText(depth_image, text_pos_x, (20, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_y, (20, 340), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        if human_vx and human_vy:
            text_pos_x = 'human VX: ' + str(human_vx)
            text_pos_y = 'human VY: ' + str(human_vy)
            cv2.putText(depth_image, text_pos_x, (20, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_y, (20, 380), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)

        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
            
        # @TODO get ORCA decision from robot_x/y, robot_vx/vy, robot_theta, human_x/y, human_vx/vy, goal_x/y

        # @TODO set robot_vy/vy & theta in ROS, and act

finally:
    # Stop streaming
    pipeline.stop()


GOT FIRST PERSON DATA AS: X:  0.06425923327508393  Y:  0.35003951806221323 

GOT FIRST PERSON DATA AS: X:  0.27411638250271514  Y:  1.9286172687501195 

GOT PERSON VELOCITY DATA AS: VX:  -0.00047805216638749587  VY:  0.000305249697141477 

GOT FIRST PERSON DATA AS: X:  -0.8668482166626913  Y:  3.1770303577283983 

GOT PERSON VELOCITY DATA AS: VX:  0.0029451461409202946  VY:  -0.010872079831080312 

GOT FIRST PERSON DATA AS: X:  -0.47731396253874264  Y:  1.8513483874892498 

GOT PERSON VELOCITY DATA AS: VX:  -0.004878679176736011  VY:  0.0175571676797008 

GOT FIRST PERSON DATA AS: X:  -1.308798397513497  Y:  4.760013324302159 

GOT PERSON VELOCITY DATA AS: VX:  -7.037138249019301e-05  VY:  8.814446970015307e-05 

GOT FIRST PERSON DATA AS: X:  -1.3863375231518373  Y:  4.756068397665997 

GOT PERSON VELOCITY DATA AS: VX:  9.108756517660016e-05  VY:  -9.426437205939636e-05 

GOT FIRST PERSON DATA AS: X:  -1.35720354464392  Y:  4.663523322486813 

GOT PERSON VELOCITY DATA AS: VX:  -2.23336

GOT FIRST PERSON DATA AS: X:  -1.4954642883999594  Y:  4.671730558405329 

GOT PERSON VELOCITY DATA AS: VX:  -0.00024717069274828566  VY:  0.0006189153758634256 

GOT FIRST PERSON DATA AS: X:  -1.5153853876883567  Y:  4.666445777171945 

GOT PERSON VELOCITY DATA AS: VX:  0.00023845876885368497  VY:  -5.50690012484183e-06 

GOT FIRST PERSON DATA AS: X:  -1.4725837239375463  Y:  4.7164584695670255 

GOT PERSON VELOCITY DATA AS: VX:  -2.127216093127244e-05  VY:  -0.0002111704883608084 

GOT FIRST PERSON DATA AS: X:  -1.5394443503433732  Y:  4.708739685263122 

GOT FIRST PERSON DATA AS: X:  -1.471697609556145  Y:  4.655417775086556 

GOT PERSON VELOCITY DATA AS: VX:  0.00019507360868086126  VY:  -0.000288261446377009 

GOT FIRST PERSON DATA AS: X:  -1.4592802688359279  Y:  4.734196809212954 

GOT PERSON VELOCITY DATA AS: VX:  0.00011481026192514849  VY:  -0.00037246676120644955 

GOT FIRST PERSON DATA AS: X:  -1.4229304139650005  Y:  4.745248925861463 

GOT FIRST PERSON DATA AS: X:  -0.299